In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%run ../load_magic/environment.py
import scrape_utils
u = scrape_utils.WebScrapingUtilities()
s = u.s
notebook_path = get_notebook_path()
print(notebook_path)
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
print(['u.{}'.format(fn) for fn in dir(u) if not fn.startswith('_')])
[fn for fn in dir() if not fn.startswith('_')]

C:\Users\dev\Documents\Repositories\job-hunting\ipynb\Indeed.com Web Spoofing.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.pickle_exists', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
['u.click_by_css', 'u.click_web_element', 'u.conjunctify_nouns', 'u.driver_get_url', 'u.enter_text_by_css', 'u.fill_in_field', 'u.fill_in_textarea', 'u.get_chrome_exe', 'u.get_contains_list', 'u.get_div', 'u.get_driver', 'u.get_dupes', 'u.get_last_sunday', 'u.get_web_element', 'u.get_web_element_text_css', 'u.indeed_url', 'u.key_in_search', 'u.log_into_indeed', 'u.move_to_element_by_css', 'u.obscure_regex', 'u.s', 'u.secrets_json', 'u.select_all', 'u.similar', 'u.unobscure_element', 'u.wait_for']


['Config', 'In', 'Out', 'RandomForestClassifier', 'exit', 'get_all_files_containing', 'get_classifier', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_modules_dataframe', 'get_notebook_path', 'get_struct_name', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'preprocess_data', 'quit', 're', 's', 'scrape_utils', 'subprocess', 'sys', 'u', 'urllib']

In [4]:

driver = u.get_driver('Chrome')

Getting the Chrome driver


In [5]:

u.driver_get_url(driver, u.indeed_url)

Getting URL: https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com%2Fjobs%3Fq%3Ddata%2Bscientist%2B%2524100%252C000%26forceLocation%3D032b3046-06a3-4876-8dfd-474eb5e7ed11%26remotejob%3D032b3046-06a3-4876-8dfd-474eb5e7ed11%26from%3Dgnav-util-jobsearch--jasx&tmpl=desktop&service=my&from=gnav-util-jobsearch--jasx&_ga=2.217171294.930897774.1613412371-1243588806.1612290936


In [6]:

name_css = '#login-email-input'
u.fill_in_field(driver, field_name='email', field_value=u.secrets_json['indeed']['email'], input_css=name_css)

Filling in the email field with dave.babbitt@gmail.com


In [7]:

password_css = '#login-password-input'
u.fill_in_field(driver, field_name='password', field_value=u.secrets_json['indeed']['password'], input_css=password_css,
                verbose=False)

In [8]:

button_css = '#login-submit-button'
button_tag = u.click_by_css(driver, button_css)

In [38]:

from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup as bs

SAVES_HTML_FOLDER = os.path.join(s.saves_folder, 'html')
jk_htmls_list = [fn.split('_')[-1] for fn in os.listdir(SAVES_HTML_FOLDER)]
PRINT_REGEX = re.compile(r'[\x9c-\x9d\uf0b7\u200b\ufb02]+')

In [45]:

from selenium.webdriver.common.action_chains import ActionChains

SPACE_REGEX = re.compile(r'[\s<>:"/\\\|\?\*_]+')
def get_listing_html(job_title_tag):
    title_str = job_title_tag.get_attribute('title')
    print(f'Clicking {title_str}')
    driver.execute_script('arguments[0].scrollIntoView();', job_title_tag)
    ActionChains(driver).move_to_element(job_title_tag).perform()
    web_element = driver.switch_to.active_element
    if web_element.get_attribute('aria-label') == 'Close':
        web_element.click()
    job_title_tag.click()
    jk_str = ''
    for iframe_tag in driver.find_elements_by_tag_name('iframe'):
        if iframe_tag.get_attribute('title') == 'Selected Job Details':
            src = iframe_tag.get_attribute('src')
            qs = urlparse(src).query
            query_dict = parse_qs(qs)
            jk_str = query_dict['jk'][0]
            driver.switch_to.frame(iframe_tag)
            break
    job_html = ''
    for job_info in driver.find_elements_by_class_name('jobsearch-JobComponent-embeddedBody'):
        job_html = job_info.get_attribute('innerHTML')
    driver.switch_to.parent_frame()
    
    return job_html, title_str, jk_str

In [43]:

driver.switch_to.parent_frame()
web_element_list = driver.find_elements_by_tag_name('a')
while web_element_list:
    print(f'Searching through {len(web_element_list)} links to find the job titles')
    for web_element in web_element_list:
        if web_element.get_attribute('data-tn-element') == 'jobTitle':
            job_html, title_str, jk_str = get_listing_html(web_element)
            if (f'{jk_str}.html' not in jk_htmls_list) and job_html:
                file_name = SPACE_REGEX.sub('_', title_str)
                file_name = f'{file_name}_{jk_str}.html'
                file_path = os.path.join(SAVES_HTML_FOLDER, file_name)
                html_str = '<html><head><title>' + title_str + '</title></head><body>'
                job_soup = bs(job_html, 'html.parser')
                with open(file_path, 'w', encoding='utf-8') as f:
                    print(html_str, file=f)
                    for div_tag in job_soup.find_all(name='div'):
                        for s in div_tag.select('template'):
                            s.extract()
                        for s in div_tag.select('script'):
                            s.extract()
                        div_str = div_tag.prettify(formatter='html')
                        div_str = PRINT_REGEX.sub('', div_str)
                        print(div_str, file=f)
                    print('</body></html>', file=f)
        elif(web_element.get_attribute('aria-label') == 'Next'):
            print('Clicking the Next link')
            web_element.click()
            web_element_list = driver.find_elements_by_tag_name('a')
            print(f'Searching through {len(web_element_list)} links to find the job titles')
            break

Searching through 312 links to find the job titles
Clicking Data Scientist, Analytics-Remote Presence


UnboundLocalError: local variable 'jk_str' referenced before assignment

In [46]:

driver.close()

In [ ]:

from IPython.display import HTML, display

for job_info in driver.find_elements_by_class_name('jobsearch-JobComponent-embeddedBody'):
    display(HTML(job_info.get_attribute('innerHTML')))